## Instructions

### 1. Install several requirements

In [ ]:
!pip3 install tensorflow
!pip3 install tensorboardX
!pip install transformers
!pip install spacy
!python -m spacy download en

### 2. Place the BERT model-related files and GloVE word embeddings following the instructions below.

* 구글 드라이브 /code 에 있는 uncased_L-12_H-768_A-12.zip 과 glove.840B.300d.zip 을 './ABSC'에 업로드 후 아래 두 코드 실행

In [3]:
!unzip uncased_L-12_H-768_A-12.zip

Archive:  uncased_L-12_H-768_A-12.zip
   creating: uncased_L-12_H-768_A-12/
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: uncased_L-12_H-768_A-12/vocab.txt  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: uncased_L-12_H-768_A-12/bert_config.json  


In [ ]:
BERT_BASE_DIR='./uncased_L-12_H-768_A-12'

!python3 convert_tf_to_pytorch/convert_tf_checkpoint_to_pytorch.py \
  --tf_checkpoint_path $BERT_BASE_DIR/bert_model.ckpt \
  --bert_config_file $BERT_BASE_DIR/bert_config.json \
  --pytorch_dump_path $BERT_BASE_DIR/pytorch_model.bin

In [10]:
!unzip './glove.840B.300d.zip' -d './datasets/glove'

Archive:  ./glove.840B.300d.zip
  inflating: ./datasets/glove/glove.840B.300d.txt  


### 3. Create dependency graphs by running following codes

* final output files are f'./{dataset_name}_datas.pkl'

In [8]:
!python generateGraph_spacy.py

2022-06-15 08:29:09.633270: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
100%|█████████████████████████████████████████| 638/638 [00:22<00:00, 28.20it/s]


In [11]:
!python data_utils.py

2022-06-15 08:47:40.796772: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
preparing twitter dataset ...
load successfully
loading word vectors ...
building embedding_matrix: 300_twitter_embedding_matrix.pkl
100%|█████████████████████████████████████████| 692/692 [00:57<00:00, 11.94it/s]
preparing rest14 dataset ...
load successfully
loading word vectors ...
building embedding_matrix: 300_rest14_embedding_matrix.pkl
100%|███████████████████████████████████████| 1120/1120 [01:31<00:00, 12.29it/s]
preparing lap14 dataset ...
load successfully
loading word vectors ...
building embedding_matrix: 300_lap14_embedding_matrix.pkl
100%|█████████████████████████████████████████| 638/638 [00:51<00:00, 12.51it/s]


In [6]:
import random
random.seed(1)
for i in range(10):
    var_name = 'seed'+str(i+1)
    locals()[var_name] = random.randint(0,10000000)

In [ ]:
!python run_classifier_word_roberta.py \
--model_name roberta_gcls \
--task_name=laptop \
--graph_type 'dg' \
--L_config_base '0,0,0,0,1,1,1,1,2,2,2,2' \
--g_config '1,1,1,1' \
--g_pooler 'att' \
--data_dir=/home/ikhyuncho23/GoBERTa/datasets/semeval14/laptops/ \
--vocab_file=/home/ikhyuncho23/GoBERTa/roberta_files/vocab.json \
--roberta_config_file=/home/ikhyuncho23/GoBERTa/roberta_files/config.json \
--max_seq_length=128 \
--train_batch_size 32 \
--eval_batch_size 32 \
--learning_rate 1.5e-5 \
--num_train_epochs 20.0 \
--local_rank -1 \
--gpu_id 0 \
--model_save_path trained_model_ckpts/ \
--para_LSR 0.2 \
--log_step 5 \
--seed $seed2